In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime


In [3]:

# =============================================================================
# 0.2 Import data
# =============================================================================

train = pd.read_csv('data/train.csv', low_memory=False)
store = pd.read_csv('data/store.csv', low_memory=False)

#%% 1. Merging store to train data

# =============================================================================
# 1.1 Since data has to has the same size as 'train', a container is created
# =============================================================================

# Creating dataframe
expanded_store = train

# =============================================================================
# 1.2 Merging variables which do not have to be changed
# =============================================================================

# Variables which can be merged right away
easy = store.loc[:,['Store','StoreType','Assortment','CompetitionDistance']]

# Variables which can be simply merged
expanded_store = pd.merge(expanded_store, easy, on=['Store'])


In [4]:
expanded=expanded_store

In [5]:
# Convert Dates to Date time
expanded['Date']=pd.to_datetime(expanded['Date'])
expanded.set_index('Date')

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance
Date,,,,,,,,,,,
2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,a,1.0,d,c,5350.0
2013-01-02,1115.0,3.0,3697.0,305.0,1.0,0.0,0,1.0,d,c,5350.0
2013-01-03,1115.0,4.0,4297.0,300.0,1.0,0.0,0,1.0,d,c,5350.0
2013-01-04,1115.0,5.0,4540.0,326.0,1.0,0.0,0,1.0,d,c,5350.0
2013-01-05,1115.0,6.0,4771.0,339.0,1.0,0.0,0,1.0,d,c,5350.0
...,...,...,...,...,...,...,...,...,...,...,...
2014-06-26,127.0,4.0,3335.0,480.0,1.0,0.0,0,0.0,d,a,1350.0
2014-06-27,127.0,5.0,4724.0,591.0,1.0,0.0,0,0.0,d,a,1350.0
2014-06-28,127.0,6.0,5398.0,716.0,1.0,0.0,0,0.0,d,a,1350.0


In [6]:
#add in variables for day of month etc
expanded['dayofweek'] = expanded['Date'].dt.dayofweek
expanded['quarter'] = expanded['Date'].dt.quarter
expanded['month'] = expanded['Date'].dt.month
expanded['year'] = expanded['Date'].dt.year
expanded['dayofyear'] = expanded['Date'].dt.dayofyear
expanded['dayofmonth'] = expanded['Date'].dt.day
expanded['weekofyear'] = expanded['Date'].dt.weekofyear

In [7]:
#create dummy variables for day of week etc and categorical variables
expanded= pd.get_dummies(expanded)
expanded= pd.get_dummies(expanded,columns=['dayofweek','dayofmonth','quarter','month'])
expanded.set_index('Date')


,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,year,dayofyear,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,1.0,5350.0,2013,1,...,0,0,0,0,0,0,0,0,0,0
2013-01-02,1115.0,3.0,3697.0,305.0,1.0,0.0,1.0,5350.0,2013,2,...,0,0,0,0,0,0,0,0,0,0
2013-01-03,1115.0,4.0,4297.0,300.0,1.0,0.0,1.0,5350.0,2013,3,...,0,0,0,0,0,0,0,0,0,0
2013-01-04,1115.0,5.0,4540.0,326.0,1.0,0.0,1.0,5350.0,2013,4,...,0,0,0,0,0,0,0,0,0,0
2013-01-05,1115.0,6.0,4771.0,339.0,1.0,0.0,1.0,5350.0,2013,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-06-26,127.0,4.0,3335.0,480.0,1.0,0.0,0.0,1350.0,2014,177,...,0,0,0,1,0,0,0,0,0,0
2014-06-27,127.0,5.0,4724.0,591.0,1.0,0.0,0.0,1350.0,2014,178,...,0,0,0,1,0,0,0,0,0,0
2014-06-28,127.0,6.0,5398.0,716.0,1.0,0.0,0.0,1350.0,2014,179,...,0,0,0,1,0,0,0,0,0,0


In [8]:
#Re-add original day of month variable etc.
expanded['dayofweek'] = expanded['Date'].dt.dayofweek
expanded['quarter'] = expanded['Date'].dt.quarter
expanded['month'] = expanded['Date'].dt.month
expanded['year'] = expanded['Date'].dt.year
expanded['dayofyear'] = expanded['Date'].dt.dayofyear
expanded['dayofmonth'] = expanded['Date'].dt.day
expanded['weekofyear'] = expanded['Date'].dt.weekofyear

expanded.set_index('Date')


,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,year,dayofyear,...,month_7,month_8,month_9,month_10,month_11,month_12,dayofweek,quarter,month,dayofmonth
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,1.0,5350.0,2013,1,...,0,0,0,0,0,0,1,1,1,1
2013-01-02,1115.0,3.0,3697.0,305.0,1.0,0.0,1.0,5350.0,2013,2,...,0,0,0,0,0,0,2,1,1,2
2013-01-03,1115.0,4.0,4297.0,300.0,1.0,0.0,1.0,5350.0,2013,3,...,0,0,0,0,0,0,3,1,1,3
2013-01-04,1115.0,5.0,4540.0,326.0,1.0,0.0,1.0,5350.0,2013,4,...,0,0,0,0,0,0,4,1,1,4
2013-01-05,1115.0,6.0,4771.0,339.0,1.0,0.0,1.0,5350.0,2013,5,...,0,0,0,0,0,0,5,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-06-26,127.0,4.0,3335.0,480.0,1.0,0.0,0.0,1350.0,2014,177,...,0,0,0,0,0,0,3,2,6,26
2014-06-27,127.0,5.0,4724.0,591.0,1.0,0.0,0.0,1350.0,2014,178,...,0,0,0,0,0,0,4,2,6,27
2014-06-28,127.0,6.0,5398.0,716.0,1.0,0.0,0.0,1350.0,2014,179,...,0,0,0,0,0,0,5,2,6,28


In [9]:
#create average sales per customer
df=expanded.groupby(by=['Store']).agg({'Sales':'sum','Customers':'sum'})
df['av_SalesPerCustomer']=df['Sales']/df['Customers']
df.sort_values(by='av_SalesPerCustomer',ascending=False)
#expanded = pd.merge(expanded, df['av_SalesPerCustomer'], on=['Store'])

#create sales per customer for each store and day
#expanded['SalesPerCustomer']=expanded['Sales']/expanded['Customers']


In [11]:
#create av sales per day of the week
#df=expanded.groupby(by=['Store','dayofweek']).agg({'Sales':'sum','Customers':'sum'})
#df


Sales  Customers
Store  dayofweek                     
1.0    0          395918.0    44318.0
       1          363963.0    42477.0
       2          346356.0    42499.0
       3          331184.0    40134.0
       4          367403.0    44650.0
...                    ...        ...
1115.0 2          407037.0    27525.0
       3          416149.0    27718.0
       4          424420.0    28989.0
       5          476643.0    35829.0
       6               0.0        0.0

[7805 rows x 2 columns]

In [10]:
#drop if any other values null
data_new=expanded
data_new=data_new.dropna(axis = 0, how ='any') 
#expanded_new.to_csv('data/train_all_10_Oct.csv')